<a href="https://colab.research.google.com/github/EdisonMaxwell/udacity_mlnd_capstone/blob/master/3_Training_and_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying urban sounds using deep learning models.
## Model Training and Evaluation
### Load preprocessed data

The data is loaded from a .npz file that contains prepocessed data that was previously created along with ordered array containing class names.
After that the data is split into training and testing sets. The amount of data is low so the splits cannot be large.

In [0]:
#installing a fork of tqdm for keras
!pip install --upgrade keras-tqdm
!pip install --upgrade tqdm

     |████████████████████████████████| 61kB 2.2MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
# Retrieve contents of the data file.
import numpy as np
from tqdm.notebook import tqdm

BASE_PATH = '/data/'
file_path = BASE_PATH+'/dataset.npz'
contents = np.load(file_path, allow_pickle=True)

X = contents['features']
y = contents['labels']
classes = contents['classes']

In [0]:
%tensorflow_version 2.x

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,shuffle=True)
#from SpectralDataGenerator import SpectralDataGenerator as gen
#train_gen = gen(audio = X_train, labels = y_train, batch_size = batch_size, dim = (width,height), channels = channels, noise_factor = 0.3)
#val_gen = gen(audio = X_val, labels = y_val, batch_size = 128, dim = (width,height), channels = channels, noise_factor = 0.3)

### Convolutional Neural Network model architecture
CNNs are some of the best tools for image classification do to their ability to quantify spatial relationships. 
Our architecture will be a relatively small `Sequential`model consisting of 3 `Conv2D`convolution layers each followed by a `MaxPooling2D` pooling layer. Finally, the output is from a `Dense` layer connected to the Convolutional.
    
The convolutional layer perform the actual feature selection. It slides a small window(aka kernek aka filter) with weights on it over the pixels of the image. It starts at the top and performs summation matrix operations and saves the result as part of an activation map. This process is called convolution. The filter parameter specifies the number of filters that will be applied to a layer in the CNN. Each layer of a CNN can have many filters. Each filter learns something differnt about the problem statment and crosses the activation threshold for different inputs.

The pooling layers reduce dimensionality wihout much loss in data. This increases robustness of the model and lowers its compute and storage requirements. Two type of pooling layers will be used in the model. Firstly, `MaxPooling2D` will be used between Convolutional layers to reduce dimensionality while preserving spatial information. 

After features are extracted, they are fed into a `GlobalAveragePooling2D` layer to flatten the features into a vector while preserving more information than simply flattening it with the `Flatten` layer.

The output layer will have 10 nodes which matches the number of target classes. The `Softmax` activation function will used on the output layer giving us a vector of probabilities.

In [0]:
%tensorflow_version 2.x

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

width = 40
height = 175
channels = 1
batch_size = 64

X_train = X_train.reshape(X_train.shape[0],width,height,channels)
X_test = X_test.reshape(X_test.shape[0],width,height,channels)

label_count = y.shape[1]

model = Sequential([
    Conv2D(filters=64,kernel_size=2,input_shape=(width,height,channels),activation='relu'),
    MaxPooling2D(pool_size=2),
      
    Conv2D(filters=64,kernel_size=2,activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.1),
    
    Conv2D(filters=128,kernel_size=2,activation='relu'),
    GlobalAveragePooling2D(),
    
    Dense(units=128,activation='tanh'),
    Dropout(0.1),
    Dense(units=label_count,activation='softmax')  
])

model.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

### Model Compilation

In [62]:
# Model architecture
model.summary()

#pretraining_accuracy
#score = model.evaluate(X_test,y_test,verbose=1)
#accuracy = 100*score[1]

#print('Pretraining accuracy:%.4f%%' % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 39, 174, 64)       320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 19, 87, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 18, 86, 64)        16448     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 43, 64)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 43, 64)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 42, 128)        32896     
_________________________________________________________________
global_average_pooling2d_1 ( (None, 128)              

### Training

The training of the model begins. CNNs are best trained on the GPU because of their high parallelization capability. One can train lower epoch numbers on the CPU but its a waste of time and energy regardless.

The model was trained for 50 epochs but they can likely be reduced.

In [63]:
%%timeit

from keras_tqdm import TQDMNotebookCallback
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, History,TensorBoard
from tensorflow.keras.optimizers import Adam
from datetime import datetime
import os

now = datetime.now()

#learning rate reduction
MAX_PATIENCE = 3

best_filepath = BASE_PATH+'/best_model.hdf5'

#callbacks
callback = [ReduceLROnPlateau(patience = MAX_PATIENCE, verbose = 0),
            ModelCheckpoint(filepath=best_filepath,monitor='val_loss',verbose=0,save_best_only=True),
            TQDMNotebookCallback(leave_outer=None,leave_inner=None)]

#compile
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

#train
model.fit(x=X_train,y=y_train,epochs=50,batch_size=batch_size,verbose=0, validation_split = 0.3, shuffle=True, callbacks=callback)

#renaming model file
#dt_string = now.strftime("%d%m%Y_%H:%M")
#new_filename = BASE_PATH+'/best_model_'+dt_string+'.hdf5'
#hos.rename(best_filepath,new_filename)


1 loop, best of 3: 3min 19s per loop


### Testing the model
We will use the small testing dataset we set aside to calulate the model's performance.

In [64]:
# Evaluating the model on the training and testing set
score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9844744
Testing Accuracy:  0.9221968


The model performs well, though the difference between the training and testing scores can indicate overfitting. This can be mitigated by augmenting the audio data before extraction of MFCCs by adding noise, distortion, pitch shifts and other random manipulation of sound.

###Making Predictions
We shall make predictions on data sourced from the outside world or online.

In [0]:
import librosa 

def get_mfccs(file_path,offset):
    max_pad_len=175

    try:
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast',offset=offset,duration=4.0) 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print(e,file_path)
        return None 
     
    return mfccs

def print_prediction(file_path,offset):
    prediction_feature = get_mfccs(file_path,offset) 
    prediction_feature = prediction_feature.reshape(1, width, height, channels)

    prediction_vector = model.predict(prediction_feature)
    predicted_class = np.argmax(prediction_vector)
    print("Prediction: ",classes[predicted_class])

    #predicted_proba_vector = model.predict_proba(prediction_feature) 
    #predicted_proba = predicted_proba_vector[0]
    #for i in range(len(predicted_proba)): 
    #    category = le.inverse_transform(np.array([i]))
    #    print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )


#Air Conditioner

In [66]:
%%time
audio_dir = '/data/sample_sounds/'
file_path = audio_dir+'air-conditioner-on-a-camping-place.wav'
print_prediction(file_path,45)

Prediction:  engine_idling
CPU times: user 214 ms, sys: 184 ms, total: 398 ms
Wall time: 426 ms


#Drilling

In [67]:
%%time
audio_dir = '/data/sample_sounds/'
file_path = 'road-drilling-and-noise.wav'
print_prediction(file_path,0)

Prediction:  drilling
CPU times: user 104 ms, sys: 115 ms, total: 218 ms
Wall time: 325 ms


#Street Music

In [68]:
%%time
audio_dir = '/data/sample_sounds/'
file_path = 'street-music.wav'
print_prediction(file_path,2)

Prediction:  street_music
CPU times: user 112 ms, sys: 116 ms, total: 228 ms
Wall time: 328 ms


##Gunshot


In [69]:
%%time
#InspectorJ (www.jshaw.co.uk) of Freesound.org
audio_dir = '/data/sample_sounds/'
file_path = 'gunshot-distant-a.wav'
print_prediction(file_path,0)

Prediction:  dog_bark
CPU times: user 104 ms, sys: 123 ms, total: 227 ms
Wall time: 321 ms


##Car Horn

In [70]:
%%time
audio_dir = '/data/sample_sounds/'
file_path = 'car-very-long-mini-countryman.wav'
print_prediction(file_path,0)

Prediction:  siren
CPU times: user 111 ms, sys: 126 ms, total: 237 ms
Wall time: 349 ms


##Dog Barking

In [71]:
%%time
#InspectorJ (www.jshaw.co.uk) of Freesound.org
audio_dir = '/data/sample_sounds/'
file_path = 'car-very-long-mini-countryman.wav'
print_prediction(file_path,1)

Prediction:  siren
CPU times: user 107 ms, sys: 115 ms, total: 222 ms
Wall time: 316 ms


#Jackhammer

In [72]:
%%time
audio_dir = '/data/sample_sounds/'
file_path = 'jackhammer.wav'
print_prediction(file_path,1)

Prediction:  jackhammer
CPU times: user 135 ms, sys: 154 ms, total: 289 ms
Wall time: 460 ms


#Observations

The model performed well on real world examples recorded with different types of hardware. I was pleasantly surprised when it was able to distinguish between drilling and a jackhammer.

I am not surprised at the fact that the model could not differentiate between an idling vehicle and an air conditioner. Those two were hard to differentiate due to similar sounds. 